# CISC 856: Reinforcement Learning
## Assignment 1: 2-Player Matrix Games
##### Hugh Morison (10179903)
##### Matthew Filipovich ()

In [3]:
from policy_iteration import p1_rewards,p2_rewards,normalize,play_game,update_policy
import matplotlib.pyplot as plt
import numpy as np

def iterate(p1, p2, matrix1, matrix2, ALPHA=0.5, T=50000, alg=1):
    p1_history = []
    p2_history = []
    for t in range(T):
        print('Iteration {}'.format(t+1))
        p1_history.append([*p1])
        p2_history.append([*p2])
        (a1, r1), (a2, r2) = play_game(matrix1, matrix2, p1, p2)
        p1 = update_policy(p1, a1, r1, ALPHA, mode=alg)
        p2 = update_policy(p2, a2, r2, ALPHA, mode=alg)
    return p1_history, p2_history

## Prisoner's Dilemma

In [4]:
p1_r = p1_rewards[0]
p2_r = p2_rewards[0]

policy_1 = normalize(*np.random.random(2))
policy_2 = normalize(*np.random.random(2))

p1_history, p2_history = iterate(policy_1, policy_2, p1_r, p2_r)

plt.figure()
x, y = zip(*p1_history)
plt.plot(x,y)

Iteration 1
2
2
2
2
Iteration 2
2
2
1
1


AssertionError: 

## Matching Pennies

In [ ]:
p1_r = p1_rewards[1]
p2_r = p2_rewards[1]

policy_1 = normalize(*np.random.random(2))
policy_2 = normalize(*np.random.random(2))

p1_history, p2_history = iterate(policy_1, policy_2, p1_r, p2_r)

## Rock Paper Scissors

In [ ]:
p1_r = p1_rewards[2]
p2_r = p2_rewards[2]

policy_1 = normalize(*np.random.random(3))
policy_2 = normalize(*np.random.random(3))

p1_history, p2_history = iterate(policy_1, policy_2, p1_r, p2_r)